In [ ]:
%run ./Includes/Classroom-Setup-02

### Other conventions

In [ ]:
# List the DA object components
print(f"Username: {da.username}")
print(f"Catalog Name: {da.catalog_name}")
print(f"Schema Name: {da.schema_name}")
print(f"Working Directory: {da.paths.working_dir}")
print(f"Dataset Location: {da.paths.datasets}")
print(f"Secondary Principal: {da.iam.secondary}")
print(f"Cluster Name: {da.cluster_name}")

#### Record the credentials

In [ ]:
# Host
# Token
# HTTP Path
DA.get_credentials_with_warehouse()

### Databricks SDF for Python

In [ ]:
import os
from databricks.sdk import WorkspaceClient

w = WorkspaceClient(
    host=os.getenv("DATABRICKS_HOST")
    , token=os.getenv("DATABRICKS_TOKEN")
)

print("Available Clusters in this Workspace:")
for c in w.Clusters.list():
    print(c.cluster_name)

In [ ]:
%sql
SELECT "${DA.schema_name}" AS Target

In [ ]:
# Verify data catalog
import os
from databricks.sdk import WorkspaceClient

w = WorkspaceClient(
    host=os.getenv("DATABRICKS_HOST")
    , token=os.getenv("DATABRICKS_TOKEN")
)

# Ensure the catalog exists
try:
    w.catalogs.get(DA.catalog_name)
    print(f"Catalog {DA.catalog_name} exists.")
except Exception as e:
    print(f"Catalog {DA.catalog_name} does not exist. Error: {e}")

In [ ]:
# Verify if schema exists
import os
from databricks.sdk import WorkspaceClient

w = WorkspaceClient(
    host=os.getenv("DATABRICKS_HOST")
    , token=os.getenv("DATABRICKS_TOKEN")
)

# Ensure the catalog exists
try:
    w.schemas.get(f"{DA.catalog_name}.{DA.schema_name}")
    print(f"Schema {DA.schema_name} exists in catalog {DA.catalog_name}.")
except Exception as e:
    print(f"Schema {DA.schema_name} does not exist in catalog {DA.catalog_name}. Error: {e}")

In [ ]:
# Declaring new table
import os
from databricks import sql

# Define table name
table_name = "sample_sdk_delta_table"

# Define the Delta table creation SQL
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {DA.catalog_name}.{DA.schema_name}.{table_name} (
    id INT,
    name STRING,
    age INT
) USING DELTA;
"""

# Use the SQL API to execute SQL Command
try:
    # Connect to Databricks SQL endpoint
    with sql.connect(
        server_hostname=os.getenv("DATABRICKS_HOST"),
        http_path=os.getenv("DATABRICKS_HTTP_PATH"),
        access_token=os.getenv("DATABRICKS_TOKEN")
    ) as conn:
        with conn.cursor() as cursor:
            # Execute the SQL command to create the table
            cursor.execute(create_table_sql)
            print(f"Delta Table {table_name} created successfully in schema '{DA.schema_name}' under catalog '{DA.catalog_name}'.")
except Exception as e:
    print(f"Error creating Delta table {table_name}: {e}")

In [ ]:
# Exploring and Managing Data Hierarchy
import os
from databricks.sdk import WorkspaceClient

# Initialize the Databricks workspace client
w = WorkspaceClient(
    host=os.getenv("DATABRICKS_HOST")
    , token=os.getenv("DATABRICKS_TOKEN")
)

# Examine the catalogs in the data hierarchy
print("Examining Metastore hierarchy:")
try:
    # List all catalogs
    catalogs = w.catalogs.list()
    for catalog in catalogs:
        catalog_name = catalog.name
        print(f"Catalog: {catalog_name}")
except Exception as e:
    print(f"Error examining data hierarchy: {e}")

In [ ]:
# Explore schema available within each catalog
import os
from databricks.sdk import WorkspaceClient

# Initialize the Databricks workspace client
w = WorkspaceClient(
    host=os.getenv("DATABRICKS_HOST")
    , token=os.getenv("DATABRICKS_TOKEN")
)

# Examine the catalogs in the data hierarchy
print("Examining Metastore hierarchy:")
try:
    # List all catalogs
    catalogs = w.catalogs.list()
    for catalog in catalogs:
        catalog_name = catalog.name
        print(f"Catalog: {catalog_name}")
        
        # List all schemas within the catalog
        schemas = w.schemas.list(catalog_name=catalog_name)
        for schema in schemas:
            schema_name = schema.name
            print(f"  Schema: {schema_name}")
except Exception as e:
    print(f"Error examining data hierarchy: {e}")

In [ ]:
# Explore the tables available with each schema
import os
from databricks.sdk import WorkspaceClient

# Initialize the Databricks workspace client
w = WorkspaceClient(
    host=os.getenv("DATABRICKS_HOST")
    , token=os.getenv("DATABRICKS_TOKEN")
)

# Examine the catalogs in the data hierarchy
print("Examining Metastore hierarchy:")
try:
    # List all catalogs
    catalogs = w.catalogs.list()
    for catalog in catalogs:
        catalog_name = catalog.name
        print(f"Catalog: {catalog_name}")
        
        # List all schemas within the catalog
        schemas = w.schemas.list(catalog_name=catalog_name)
        for schema in schemas:
            schema_name = schema.name
            print(f"  Schema: {schema_name}")
            
            # List all tables within the schema
            tables = w.tables.list(catalog_name=catalog_name, schema_name=schema_name)
            for table in tables:
                table_name = table.name
                print(f"    Table: {table_name}")
except Exception as e:
    print(f"Error examining data hierarchy: {e}")

#### Cleanup

In [ ]:
%sh rm -rf ./var

In [ ]:
import os
from databricks import sql

# Define the Delta table name
table_name = "sample_sdk_delta_table"

# Define the Delta table drop SQL
drop_table_sql = f"""
DROP TABLE IF EXISTS {DA.catalog_name}.{DA.schema_name}.{table_name};
"""

# Use the SQL API to execute SQL Command
try:
    # Connect to Databricks SQL endpoint
    with sql.connect(
        server_hostname=os.getenv("DATABRICKS_HOST"),
        http_path=os.getenv("DATABRICKS_HTTP_PATH"),
        access_token=os.getenv("DATABRICKS_TOKEN")
    ) as conn:
        with conn.cursor() as cursor:
            # Execute the SQL command to drop the table
            cursor.execute(drop_table_sql)
            print(f"Delta Table {table_name} dropped successfully from schema '{DA.schema_name}' under catalog '{DA.catalog_name}'.")
except Exception as e:
    print(f"Error dropping Delta table {table_name}: {e}")